In [1]:
import sys, pathlib
import pandas as pd
import numpy as np
import joblib

PROJECT_ROOT = pathlib.Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg
import src.evol_utils as eu


In [2]:
# Cargar precios y calcular retornos diarios
df_prices = pd.read_parquet(cfg.DATA / 'raw' / 'prices.parquet').sort_index()
df_ret = np.log(df_prices / df_prices.shift(1)).dropna()
print('✅ Datos cargados:', df_ret.shape)


In [3]:
def rebalancear_markowitz(fecha, df_ret, w_prev=None):
    idx = df_ret.index.get_loc(fecha)
    ventana = df_ret.iloc[idx-cfg.WINDOW: idx]
    mu_hat = ventana.mean().values
    Sigma = ventana.cov().values
    res = eu.resolver_optimizacion(mu_hat, Sigma, w_prev=w_prev)
    w_star = eu.elegir_w_star(res, mu_hat, Sigma, w_prev=w_prev)
    diarios = df_ret.iloc[idx: idx+cfg.REBAL_FREQ].dot(w_star)
    return w_star, diarios


In [4]:
fechas = df_ret.loc[cfg.START_BACKTEST:].index
w_prev = None
series = []
for i in range(cfg.WINDOW, len(fechas)-cfg.REBAL_FREQ, cfg.REBAL_FREQ):
    fecha = fechas[i]
    w_prev, ret_d = rebalancear_markowitz(fecha, df_ret, w_prev)
    series.append(ret_d)
    print(f'✅ {fecha.date()} | {len(ret_d)} días')


In [5]:
serie = pd.concat(series)
joblib.dump(serie, cfg.RESULT / 'backtest_markowitz.pkl')
print('✅ Serie guardada:', cfg.RESULT / 'backtest_markowitz.pkl')
